In [4]:
# 5th



import math

# ----- Input -----
labels = ["Cat", "Dog", "Rabbit"]
confusion_matrix = [
    [5, 10, 5],   # Predicted Cat
    [15, 20, 10], # Predicted Dog
    [0, 15, 10]   # Predicted Rabbit
]

# ----- Functions -----
def compute_metrics(labels, cm):
    n = len(labels)

    # Column sums = actual totals, Row sums = predicted totals
    col_sums = [sum(cm[r][c] for r in range(n)) for c in range(n)]
    row_sums = [sum(cm[r]) for r in range(n)]

    results = {}
    for i, label in enumerate(labels):
        tp = cm[i][i]
        fp = row_sums[i] - tp
        fn = col_sums[i] - tp

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0

        results[label] = {"precision": precision, "recall": recall}
    return results

def macro_average(metrics):
    precisions = [m["precision"] for m in metrics.values()]
    recalls = [m["recall"] for m in metrics.values()]
    return {
        "precision": sum(precisions) / len(precisions),
        "recall": sum(recalls) / len(recalls),
    }

def micro_average(cm):
    tp_sum = sum(cm[i][i] for i in range(len(cm)))
    total = sum(sum(row) for row in cm)
    micro = tp_sum / total if total > 0 else 0.0
    return {"precision": micro, "recall": micro}

def fmt(x):
    return f"{x:.4f}"

# ----- Main -----
metrics = compute_metrics(labels, confusion_matrix)
macro = macro_average(metrics)
micro = micro_average(confusion_matrix)

print("Per-class Precision and Recall:")
for label, vals in metrics.items():
    print(f"  {label}: Precision = {fmt(vals['precision'])}, Recall = {fmt(vals['recall'])}")

print("\nMacro-averaged:")
print(f"  Precision = {fmt(macro['precision'])}, Recall = {fmt(macro['recall'])}")

print("\nMicro-averaged:")
print(f"  Precision = {fmt(micro['precision'])}, Recall = {fmt(micro['recall'])}")


Per-class Precision and Recall:
  Cat: Precision = 0.2500, Recall = 0.2500
  Dog: Precision = 0.4444, Recall = 0.4444
  Rabbit: Precision = 0.4000, Recall = 0.4000

Macro-averaged:
  Precision = 0.3648, Recall = 0.3648

Micro-averaged:
  Precision = 0.3889, Recall = 0.3889


In [5]:
# 8th


from collections import Counter, defaultdict
from typing import List, Tuple, Dict

# ------------------------------
# 1) Training corpus
# ------------------------------
corpus_lines = [
    "<s> I love NLP </s>",
    "<s> I love deep learning </s>",
    "<s> deep learning is fun </s>",
]

def tokenize(line: str) -> List[str]:
    """Split a sentence on whitespace (keep <s> and </s>)."""
    return line.split()

# ------------------------------
# 2) Unigram & Bigram counts
# ------------------------------
def get_counts(corpus: List[str]) -> Tuple[Counter, Counter, Dict[str, int]]:
    """
    Returns:
      - unigrams: Counter of tokens
      - bigrams: Counter of (w1, w2) pairs
      - next_totals: dict mapping w1 -> total times any w2 follows w1
    """
    unigrams = Counter()
    bigrams = Counter()
    next_totals = defaultdict(int)  # sum_x count(w_{i-1}, x) for each history word

    for line in corpus:
        toks = tokenize(line)
        unigrams.update(toks)
        for i in range(len(toks) - 1):
            w1, w2 = toks[i], toks[i + 1]
            bigrams[(w1, w2)] += 1
            next_totals[w1] += 1

    return unigrams, bigrams, next_totals

# ------------------------------
# 3) Bigram MLE
# ------------------------------
def bigram_mle_prob(w2: str, w1: str, bigrams: Counter, next_totals: Dict[str, int]) -> float:
    """
    MLE: P(w2 | w1) = count(w1, w2) / sum_x count(w1, x)
    Returns 0.0 if w1 never appears (no outgoing bigrams).
    """
    num = bigrams.get((w1, w2), 0)
    den = next_totals.get(w1, 0)
    return (num / den) if den > 0 else 0.0

# ------------------------------
# 4) Sentence probability (product of bigrams)
# ------------------------------
def sentence_probability(sentence: str, bigrams: Counter, next_totals: Dict[str, int], verbose: bool = True) -> float:
    """
    Computes P(w1,...,wk) = Π_i P(w_i | w_{i-1}) with bigram MLE.
    If verbose=True, prints the step-by-step conditional probabilities.
    """
    toks = tokenize(sentence)
    p = 1.0
    steps = []
    for i in range(len(toks) - 1):
        w1, w2 = toks[i], toks[i + 1]
        prob = bigram_mle_prob(w2, w1, bigrams, next_totals)
        steps.append((w1, w2, prob))
        p *= prob

    if verbose:
        print(f"\nSentence: {sentence}")
        for w1, w2, prob in steps:
            print(f"  P({w2}|{w1}) = {prob:.6f}")
        print(f"  => Sentence probability = {p:.6f}")
    return p

# ------------------------------
# 5) Driver
# ------------------------------
def main():
    print("=== Bigram Language Model (MLE) on the 'I love NLP' corpus ===")
    print("Theory note: P(w_i | w_{i-1}) = count(w_{i-1}, w_i) / sum_x count(w_{i-1}, x)")
    print("We multiply these bigram probabilities across a sentence to get its probability.\n")

    uni, bi, next_totals = get_counts(corpus_lines)

    # Show compact counts useful for the report
    print("Unigram counts (subset):")
    for tok in ["<s>", "I", "love", "NLP", "deep", "learning", "is", "fun", "</s>"]:
        print(f"  {tok}: {uni.get(tok, 0)}")

    print("\nSome Bigram counts:")
    show_pairs = [
        ("<s>", "I"), ("<s>", "deep"),
        ("I", "love"),
        ("love", "NLP"), ("love", "deep"),
        ("deep", "learning"),
        ("learning", "</s>"), ("learning", "is"),
        ("is", "fun"),
        ("fun", "</s>"),
        ("NLP", "</s>"),
    ]
    for w1, w2 in show_pairs:
        print(f"  count({w1!r},{w2!r}) = {bi.get((w1, w2), 0)}; total after {w1!r} = {next_totals.get(w1, 0)}")

    # Test sentences
    s1 = "<s> I love NLP </s>"
    s2 = "<s> I love deep learning </s>"

    p1 = sentence_probability(s1, bi, next_totals, verbose=True)
    p2 = sentence_probability(s2, bi, next_totals, verbose=True)

    # Decision + explanation
    print("\n=== Decision ===")
    if p1 > p2:
        print("Model prefers S1 =", s1)
        print(f"Reason: P(S1) = {p1:.6f} is greater than P(S2) = {p2:.6f}.")
        print("Why: both paths share early factors, but S2 ends with P(</s>|learning)=1/2 while S1 ends with P(</s>|NLP)=1.")
    elif p2 > p1:
        print("Model prefers S2 =", s2)
        print(f"Reason: P(S2) = {p2:.6f} is greater than P(S1) = {p1:.6f}.")
    else:
        print("Model is indifferent: P(S1) = P(S2) =", f"{p1:.6f}")

if __name__ == "__main__":
    main()


=== Bigram Language Model (MLE) on the 'I love NLP' corpus ===
Theory note: P(w_i | w_{i-1}) = count(w_{i-1}, w_i) / sum_x count(w_{i-1}, x)
We multiply these bigram probabilities across a sentence to get its probability.

Unigram counts (subset):
  <s>: 3
  I: 2
  love: 2
  NLP: 1
  deep: 2
  learning: 2
  is: 1
  fun: 1
  </s>: 3

Some Bigram counts:
  count('<s>','I') = 2; total after '<s>' = 3
  count('<s>','deep') = 1; total after '<s>' = 3
  count('I','love') = 2; total after 'I' = 2
  count('love','NLP') = 1; total after 'love' = 2
  count('love','deep') = 1; total after 'love' = 2
  count('deep','learning') = 2; total after 'deep' = 2
  count('learning','</s>') = 1; total after 'learning' = 2
  count('learning','is') = 1; total after 'learning' = 2
  count('is','fun') = 1; total after 'is' = 1
  count('fun','</s>') = 1; total after 'fun' = 1
  count('NLP','</s>') = 1; total after 'NLP' = 1

Sentence: <s> I love NLP </s>
  P(I|<s>) = 0.666667
  P(love|I) = 1.000000
  P(NLP|love)